In [1]:
import folium
import pandas as pd
import branca

In [2]:
#read csv with bus data
df = pd.read_csv('../data/vehicle-position-matched-offline-20180813.csv')
#split the linkref into wayid, source and target columns
df['wayId'], df['source'], df['target'] = df['LinkRef'].str.split(':').str

#group the data by wayId and calculate the mean and count
df_agg = df.groupby('wayId', as_index=False)['Speed'].agg({"mean","count"}).reset_index()

df_agg.head()

,wayId,mean,count
0,103524893,11.613532,3995
1,103524901,0.866864,118
2,103524902,7.591159,15137
3,108233932,7.243998,973
4,108233935,5.323023,1396


In [3]:
#Create two colormaps, one for the mean speed and one for the number of meassurements
cm_mean = branca.colormap.linear.YlOrRd_04.scale(0,df_agg['mean'].max())
cm_count = branca.colormap.linear.YlOrRd_04.scale(0,df_agg['count'].max())

#Create a map object in valby
m = folium.Map(location=[55.6691591,12.5103835], zoom_start=16)


#Define two style functions, that change the color depending on the value of the mean speed/count
def style_trafic_mean(feature):
    wayid = feature['properties']['WayId']
    val = df_agg[df_agg['wayId']==str(wayid)]['mean']
    if len(val)==0:
        val = 0
    else:
        val = val.values[0]
        
    return {
            'fillOpacity': 0.5,
            'weight': 10,
            'color':cm_mean(val)
        }

def style_trafic_count(feature):
    wayid = feature['properties']['WayId']
    val = df_agg[df_agg['wayId']==str(wayid)]['count']
    if len(val)==0:
        val = 0
    else:
        val = val.values[0]
    
    return {
            'fillOpacity': 0.5,
            'weight': 10,
            'color':cm_count(val)
    }


#Add the geojson with colors
folium.GeoJson('../data/road_network.geojson',name='mean speed', style_function=style_trafic_mean).add_to(m)

#This is a hack to show a colorbar, and also include all the streets
m.choropleth(geo_data='../data/road_network.geojson'
             , data=df_agg, columns=['wayId','mean']
             , key_on='feature.properties.WayId'
             , fill_color='YlOrRd'
             , name='Streets'
             , legend_name='Mean speed [m/s]')
#Add a control layer to our map
folium.LayerControl().add_to(m)

#Show the resulting map
m


In [4]:
#Do the same as before, but now look at the meassurement density instead

m2 = folium.Map(location=[55.6691591,12.5103835], zoom_start=16)

folium.GeoJson('../data/road_network.geojson',name='measurement density', style_function=style_trafic_count).add_to(m2)

m2.choropleth(geo_data='../data/road_network.geojson'
             , data=df_agg, columns=['wayId','count']
             , key_on='feature.properties.WayId'
             , fill_color='YlOrRd'
             , name='Streets'
             , legend_name='Meassurements ')

folium.LayerControl().add_to(m2)

m2